In [1]:
from kmcluster.core.data import sparse_to_mat
Pt_H1_links = [
    [0, 1, 0.15],
    [0, 2, 0.61],
    [0, 3, 0.39],
    [2, 4, 0.27],
    [2, 6, 0.50],
    [2, 8, 0.66],
    [3, 8, 0.50],
    [5, 7, 0.52],
    [5, 9, 0.66],
    [5, 6, 0.66],
]
H_H1_links = [
    [0, 6, 0.52],
    [1, 2, 0.28],
    [2, 5, 0.22],
    [3, 5, 0.22],
    [7, 8, 0.15],
    [8, 9, 0.14],
]
Pt4H1_rawE = [
    -17.71720725,
    -17.68531409,
    -17.57336808,
    -17.50640668,
    -17.50097929,
    -17.50887522,
    -17.38155630,
    -17.25580137,
    -17.15164472,
    -17.13649884,
]
H1_E = [Pt4H1_rawE[x] - Pt4H1_rawE[0] for x in range(0, len(Pt4H1_rawE))]

Pt4H1_rev = []
for i in range(0, len(Pt_H1_links)):
    Pt4H1_rev.append(
        [
            Pt_H1_links[i][1],
            Pt_H1_links[i][0],
            round(
                (Pt_H1_links[i][2] + H1_E[Pt_H1_links[i][0]]) - H1_E[Pt_H1_links[i][1]],
                2,
            ),
        ]
    )

# raw energies
Pt4H1_rawE = [
    -17.71720725,
    -17.68531409,
    -17.57336808,
    -17.50640668,
    -17.50097929,
    -17.50887522,
    -17.38155630,
    -17.25580137,
    -17.15164472,
    -17.13649884,
]

# relative energies
H1_E = [Pt4H1_rawE[x] - Pt4H1_rawE[0] for x in range(0, len(Pt4H1_rawE))]

# all the forward barriers
Pt_H1_links = [
    [0, 1, 0.15],
    [0, 2, 0.61],
    [0, 3, 0.39],
    [2, 4, 0.27],
    [2, 6, 0.50],
    [2, 8, 0.66],
    [3, 8, 0.55],
    [5, 7, 0.52],
    [5, 9, 0.66],
    [5, 6, 0.66],
    [0, 6, 0.52],
    [1, 2, 0.28],
    [2, 5, 0.22],
    [3, 5, 0.22],
    [7, 8, 0.15],
    [8, 9, 0.14],
]

# calculating the reverse barriers
Pt4H1_rev = []
for i in range(0, len(Pt_H1_links)):
    Pt4H1_rev.append(
        [
            Pt_H1_links[i][1],
            Pt_H1_links[i][0],
            round(
                (Pt_H1_links[i][2] + H1_E[Pt_H1_links[i][0]]) - H1_E[Pt_H1_links[i][1]],
                2,
            ),
        ]
    )

# all barriers
Pt_H1_all = Pt_H1_links + Pt4H1_rev
energies_mat = sparse_to_mat(Pt_H1_all)

In [2]:
energies_mat

array([[0.  , 0.15, 0.61, 0.39, 0.  , 0.  , 0.52, 0.  , 0.  , 0.  ],
       [0.12, 0.  , 0.28, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.47, 0.17, 0.  , 0.  , 0.27, 0.22, 0.5 , 0.  , 0.66, 0.  ],
       [0.18, 0.  , 0.  , 0.  , 0.  , 0.22, 0.  , 0.  , 0.55, 0.  ],
       [0.  , 0.  , 0.2 , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.16, 0.22, 0.  , 0.  , 0.66, 0.52, 0.  , 0.66],
       [0.18, 0.  , 0.31, 0.  , 0.  , 0.53, 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.27, 0.  , 0.  , 0.15, 0.  ],
       [0.  , 0.  , 0.24, 0.2 , 0.  , 0.  , 0.  , 0.05, 0.  , 0.14],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.29, 0.  , 0.  , 0.12, 0.  ]])

In [5]:
from kmcluster.core.kmc_minimum import kmc 
from kmcluster.core.transition_conditions import rfkmc
from kmcluster.core.intialize import random_init, boltz
from kmcluster.core.data import sparse_to_mat

T_kelvin = 100
temp_boltz = T_kelvin * 8.617 * 10 ** (-5)
rfkmc_obj = rfkmc(k_b_t=temp_boltz, energy_mat=energies_mat)
time_stop = 0.0001
init_boltz = boltz(energies=H1_E, T=temp_boltz, size=10000)
init_random = random_init(10000, energies_mat.shape[0])
# initialize kmc object
kmc_boltz = kmc(
    time_stop=time_stop,
    energies=energies_mat,
    draw_crit=rfkmc_obj,
    initialization=init_random,
    checkpoint=True,
    sample_frequency=1001,
    ckptprefix="Pt4H2_g_{}_".format(T_kelvin),
    checkpoint_dir="./checkpoints/", # change this to organize runs
    batch_size=1, 
)

# run calcs
# can either run to a certain number of steps or until a certain time
# n_steps=-1 means run until time_stop is reached
kmc_boltz.run(n_steps=-1)
# kmc_boltz.run(n_steps=100)
trajectories = kmc_boltz.trajectories

done initializing....
starting run
hit checkpoint 1/10
saving checkpoint at step 13
hit checkpoint 2/10
saving checkpoint at step 14
hit checkpoint 3/10
saving checkpoint at step 17
hit checkpoint 4/10
saving checkpoint at step 22
hit checkpoint 5/10
saving checkpoint at step 23
hit checkpoint 6/10
saving checkpoint at step 27
hit checkpoint 7/10
saving checkpoint at step 31
hit checkpoint 8/10
saving checkpoint at step 32
hit checkpoint 9/10
saving checkpoint at step 33
Calc Done!
hit checkpoint 10/10
saving checkpoint at step 34
done with kmc run to stop time 0.0001
this took 34 steps
sum of results at each probe time: [10000. 10000. 10000. ... 10000. 10000.     0.]
Lowest time at final step 34: 1.00005e-04
mean time at final step: 7.70416e-04



In [6]:
kmc_boltz.plot_top_n_states_stacked(
    n_show = -1,
    max_time=0.0001, 
    title="State Distribution, {}K".format(T_kelvin),
    xlabel="Time (s)",
    ylabel="Population Proportion", 
    save=True, 
    show=True,
    save_name="./plots/Pt4H2_stacked_{}_top{}.png".format(T_kelvin, -1)
)